In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [3]:
!unzip train_test.zip -d /content/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/train/TRAIN_95000.png  
  inflating: /content/train/TRAIN_95001.png  
  inflating: /content/train/TRAIN_95002.png  
  inflating: /content/train/TRAIN_95003.png  
  inflating: /content/train/TRAIN_95004.png  
  inflating: /content/train/TRAIN_95005.png  
  inflating: /content/train/TRAIN_95006.png  
  inflating: /content/train/TRAIN_95007.png  
  inflating: /content/train/TRAIN_95008.png  
  inflating: /content/train/TRAIN_95009.png  
  inflating: /content/train/TRAIN_95010.png  
  inflating: /content/train/TRAIN_95011.png  
  inflating: /content/train/TRAIN_95012.png  
  inflating: /content/train/TRAIN_95013.png  
  inflating: /content/train/TRAIN_95014.png  
  inflating: /content/train/TRAIN_95015.png  
  inflating: /content/train/TRAIN_95016.png  
  inflating: /content/train/TRAIN_95017.png  
  inflating: /content/train/TRAIN_95018.png  
  inflating: /content/train/TRAIN_95019.png  
  inflating: /content/train/TRAIN_95020.png 

In [4]:
# deep-text-benchmark 다운로드

# %cd "/content/drive/MyDrive"
# !git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

!pip install torch torchvision
!pip install lmdb pillow nltk natsort
!pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=2bb72276cf25b861f12f4d50e1f6945cf2601dbb11e79c2f32791bcd9e25dd17
  Stored in directory: /root/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


In [5]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [6]:
train_df

,id,img_path,label
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색
1,TRAIN_00001,./train/TRAIN_00001.png,머
2,TRAIN_00002,./train/TRAIN_00002.png,차차
3,TRAIN_00003,./train/TRAIN_00003.png,써
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다
...,...,...,...
556290,TRAIN_556290,./train/TRAIN_556290.png,푿
556291,TRAIN_556291,./train/TRAIN_556291.png,푿
556292,TRAIN_556292,./train/TRAIN_556292.png,푿
556293,TRAIN_556293,./train/TRAIN_556293.png,푿


In [7]:
test_df

,id,img_path
0,TEST_00000,./test/TEST_00000.png
1,TEST_00001,./test/TEST_00001.png
2,TEST_00002,./test/TEST_00002.png
3,TEST_00003,./test/TEST_00003.png
4,TEST_00004,./test/TEST_00004.png
...,...,...
74116,TEST_74116,./test/TEST_74116.png
74117,TEST_74117,./test/TEST_74117.png
74118,TEST_74118,./test/TEST_74118.png
74119,TEST_74119,./test/TEST_74119.png


In [8]:
from sklearn.model_selection import train_test_split

X_test = test_df["img_path"].values

X = train_df["img_path"].values
y = train_df["label"].values
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.2, random_state=0)
len(X_test),len(X_train),len(X_val),len(y_train),len(y_val)

(74121, 445036, 111259, 445036, 111259)

In [9]:
# 학습용 gt.txt파일 생성

# %cd /content/drive/MyDrive/
# with open("train_gt.txt", "w") as f:
#   for i in range(len(X_train)):
#     f.write(X_train[i]+"    "+y_train[i]+"\n")

# with open("val_gt.txt", "w") as f:
#   for i in range(len(X_test)):
#     f.write(X_val[i]+"    "+y_val[i]+"\n")

# with open("test_gt.txt", "w") as f:
#   for i in range(len(X_train)):
#     f.write(X_train[i]+"    "+y_train[i]+"\n")

In [11]:
# %cd "/content/drive/MyDrive/deep-text-recognition-benchmark/"
# # create_lmdb_dataset.py의 "imagePath, label = datalist[i].strip('\n').split('\t')"에서 \t가 여섯칸을 차지하는
# # 현상때문에 직접 "    "으로 빈칸 4개를 기준으로 split했습니다.

# !python create_lmdb_dataset.py \
#         --inputPath /content/ \
#         --gtFile /content/drive/MyDrive/train_gt.txt \
#         --outputPath data_lmdb/train/

# !python create_lmdb_dataset.py \
#         --inputPath /content/ \
#         --gtFile /content/drive/MyDrive/val_gt.txt \
#         --outputPath data_lmdb/val/

In [10]:
# deep-text-recognition-benchmark 의 train.py --character를 다음과 같이 바꾼다

" !\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~가각간갇갈감갑값강갖같갚갛개객걀거걱건걷걸검겁것겉게겨격겪견결겹경곁계고곡곤곧골곰곱곳공과관광괜괴굉교구국군굳굴굵굶굽궁권귀규균그극근글긁금급긋긍기긴길김깅깊까깎깐깔깜깝깥깨꺼꺾껍껏껑께껴꼬꼭꼴꼼꼽꽂꽃꽉꽤꾸꿀꿈뀌끄끈끊끌끓끔끗끝끼낌나낙낚난날낡남납낫낭낮낯낱낳내냄냉냐냥너넉널넓넘넣네넥넷녀녁년념녕노녹논놀놈농높놓놔뇌뇨누눈눕뉘뉴늄느늑는늘늙능늦늬니닐님다닥닦단닫달닭닮담답닷당닿대댁댐더덕던덜덤덥덧덩덮데델도독돈돌돕동돼되된두둑둘둠둡둥뒤뒷드득든듣들듬듭듯등디딩딪따딱딴딸땀땅때땜떠떡떤떨떻떼또똑뚜뚫뚱뛰뜨뜩뜯뜰뜻띄라락란람랍랑랗래랜램랫략량러럭런럴럼럽럿렁렇레렉렌려력련렬렵령례로록론롬롭롯료루룩룹룻뤄류륙률륭르른름릇릎리릭린림립릿마막만많말맑맘맙맛망맞맡맣매맥맨맵맺머먹먼멀멈멋멍멎메멘멩며면멸명몇모목몰몸몹못몽묘무묵묶문묻물뭄뭇뭐뭣므미민믿밀밉밌및밑바박밖반받발밝밟밤밥방밭배백뱀뱃뱉버번벌범법벗베벤벼벽변별볍병볕보복볶본볼봄봇봉뵈뵙부북분불붉붐붓붕붙뷰브블비빌빗빚빛빠빨빵빼뺨뻐뻔뻗뼈뽑뿌뿐쁘쁨사삭산살삶삼상새색샌생서석섞선설섬섭섯성세센셈셋션소속손솔솜솟송솥쇄쇠쇼수숙순술숨숫숲쉬쉽슈스슨슬슴습슷승시식신싣실싫심십싱싶싸싹쌀쌍쌓써썩썰썹쎄쏘쏟쑤쓰쓸씀씌씨씩씬씹씻아악안앉않알앓암압앗앙앞애액야약얇양얗얘어억언얹얻얼엄업없엇엉엌엎에엔엘여역연열엷염엽엿영옆예옛오옥온올옮옳옷와완왕왜왠외왼요욕용우욱운울움웃웅워원월웨웬위윗유육율으윽은을음응의이익인일읽잃임입잇있잊잎자작잔잖잘잠잡장잦재쟁저적전절젊점접젓정젖제젠젯져조족존졸좀좁종좋좌죄주죽준줄줌줍중쥐즈즉즌즐즘증지직진질짐집짓징짙짚짜짝짧째쨌쩌쩍쩐쪽쫓쭈쭉찌찍찢차착찬찮찰참창찾채책챔챙처척천철첫청체쳐초촉촌총촬최추축춘출춤춥춧충취츠측츰층치칙친칠침칭카칸칼캐캠커컨컬컴컵컷켓켜코콜콤콩쾌쿠퀴크큰클큼키킬타탁탄탈탑탓탕태택탤터턱털텅테텍텔템토톤톱통퇴투툼퉁튀튜트특튼튿틀틈티틱팀팅파팎판팔패팩팬퍼퍽페펴편펼평폐포폭표푸푹풀품풍퓨프플픔피픽필핏핑하학한할함합항해핵핸햄햇행향허헌험헤헬혀현혈협형혜호혹혼홀홍화확환활황회획횟효후훈훌훔훨휘휴흉흐흑흔흘흙흡흥흩희흰히힘"

' !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~가각간갇갈감갑값강갖같갚갛개객걀거걱건걷걸검겁것겉게겨격겪견결겹경곁계고곡곤곧골곰곱곳공과관광괜괴굉교구국군굳굴굵굶굽궁권귀규균그극근글긁금급긋긍기긴길김깅깊까깎깐깔깜깝깥깨꺼꺾껍껏껑께껴꼬꼭꼴꼼꼽꽂꽃꽉꽤꾸꿀꿈뀌끄끈끊끌끓끔끗끝끼낌나낙낚난날낡남납낫낭낮낯낱낳내냄냉냐냥너넉널넓넘넣네넥넷녀녁년념녕노녹논놀놈농높놓놔뇌뇨누눈눕뉘뉴늄느늑는늘늙능늦늬니닐님다닥닦단닫달닭닮담답닷당닿대댁댐더덕던덜덤덥덧덩덮데델도독돈돌돕동돼되된두둑둘둠둡둥뒤뒷드득든듣들듬듭듯등디딩딪따딱딴딸땀땅때땜떠떡떤떨떻떼또똑뚜뚫뚱뛰뜨뜩뜯뜰뜻띄라락란람랍랑랗래랜램랫략량러럭런럴럼럽럿렁렇레렉렌려력련렬렵령례로록론롬롭롯료루룩룹룻뤄류륙률륭르른름릇릎리릭린림립릿마막만많말맑맘맙맛망맞맡맣매맥맨맵맺머먹먼멀멈멋멍멎메멘멩며면멸명몇모목몰몸몹못몽묘무묵묶문묻물뭄뭇뭐뭣므미민믿밀밉밌및밑바박밖반받발밝밟밤밥방밭배백뱀뱃뱉버번벌범법벗베벤벼벽변별볍병볕보복볶본볼봄봇봉뵈뵙부북분불붉붐붓붕붙뷰브블비빌빗빚빛빠빨빵빼뺨뻐뻔뻗뼈뽑뿌뿐쁘쁨사삭산살삶삼상새색샌생서석섞선설섬섭섯성세센셈셋션소속손솔솜솟송솥쇄쇠쇼수숙순술숨숫숲쉬쉽슈스슨슬슴습슷승시식신싣실싫심십싱싶싸싹쌀쌍쌓써썩썰썹쎄쏘쏟쑤쓰쓸씀씌씨씩씬씹씻아악안앉않알앓암압앗앙앞애액야약얇양얗얘어억언얹얻얼엄업없엇엉엌엎에엔엘여역연열엷염엽엿영옆예옛오옥온올옮옳옷와완왕왜왠외왼요욕용우욱운울움웃웅워원월웨웬위윗유육율으윽은을음응의이익인일읽잃임입잇있잊잎자작잔잖잘잠잡장잦재쟁저적전절젊점접젓정젖제젠젯져조족존졸좀좁종좋좌죄주죽준줄줌줍중쥐즈즉즌즐즘증지직진질짐집짓징짙짚짜짝짧째쨌쩌쩍쩐쪽쫓쭈쭉찌찍찢차착찬찮찰참창찾채책챔챙처척천철첫청체쳐초촉촌총촬최추축춘출춤춥춧충취츠측츰층치칙친칠침칭카칸칼캐캠커컨컬컴컵컷켓켜코콜콤콩쾌쿠퀴크큰클큼키킬타탁탄탈탑탓탕태택탤터턱털텅테텍텔템토톤톱통퇴투툼퉁튀튜트특튼튿틀틈티틱팀팅파팎판팔패팩팬퍼퍽페펴편펼평폐포폭표푸푹풀품풍퓨프플픔피픽필핏핑하학한할함합항해핵핸햄햇행향허헌험헤헬혀현혈협형혜호혹혼홀홍화확환활황회획횟효후훈훌훔훨휘휴흉흐

In [12]:
# https://www.jaided.ai/easyocr/modelhub/  에서 한국어 pretrained_model을 다운받는다.

# %cd /content/drive/MyDrive
# !unzip korean_g2.zip -d /content/drive/MyDrive/deep-text-recognition-benchmark/pretrained_model

In [26]:
 %cd /content/drive/MyDrive/deep-text-recognition-benchmark
# 파이썬 버전차이인가 다음과 같은 오류가 난다.
# 
# File "/content/drive/MyDrive/deep-text-recognition-benchmark/dataset.py", line 87, in get_batch
#     image, text = data_loader_iter.next()
# AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'
# 
# image, text = next(data_loader_iter)로 바꿔주었다.


# File "/content/drive/MyDrive/deep-text-recognition-benchmark/dataset.py", line 92, in get_batch
#     image, text = self.dataloader_iter_list[i].next()
# AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'
#
# 이것도 image, text = next(self.dataloader_iter_list[i])로 바꿔주었다.


 !python3 train.py \
        --train_data "data_lmdb/train/" \
        --valid_data "data_lmdb/val/" \
        --select_data / \
        --batch_ratio 1 \
        --Transformation None \
        --FeatureExtraction "VGG" \
        --SequenceModeling "BiLSTM" \
        --Prediction "CTC" \
        --input_channel 1 \
        --output_channel 256 \
        --hidden_size 256 \
        --saved_model "pretrained_model/korean_g2.pth" \
        --FT

/content/drive/MyDrive/deep-text-recognition-benchmark
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data_lmdb/train/
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    data_lmdb/train/	 dataset: /
sub-directory:	/.	 num samples: 151426
num total samples of /: 151426 x 1.0 (total_data_usage_ratio) = 151426
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow o

In [28]:
 %cd /content/drive/MyDrive/deep-text-recognition-benchmark

!python3 demo.py \
        --Transformation None \
        --FeatureExtraction "VGG" \
        --SequenceModeling "BiLSTM" \
        --Prediction "CTC" \
        --input_channel 1 \
        --output_channel 256 \
        --hidden_size 256 \
        --image_folder /content/test/ \
        --saved_model saved_models/다른데서학습하던거.pth

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/test/TEST_69199.png	호과적                      	0.8636
/content/test/TEST_69200.png	운동장                      	0.9994
/content/test/TEST_69201.png	요류                       	0.6137
/content/test/TEST_69202.png	운동장                      	0.9994
/content/test/TEST_69203.png	판배                       	0.1634
/content/test/TEST_69204.png	잠                        	1.0000
/content/test/TEST_69205.png	일치                       	0.9958
/content/test/TEST_69206.png	취소하다                     	0.5318
/content/test/TEST_69207.png	운행                       	0.9089
/content/test/TEST_69208.png	금지                       	0.9999
/content/test/TEST_69209.png	스키장                      	0.8176
/content/test/TEST_69210.png	불출                       	0.2886
/content/test/TEST_69211.png	닮다                       	0.7776
/content/test/TEST_69212.png	부르다                      	0.9722
/content/test/TEST_69213.png	끝나다                      	0.9980
/content/test/TEST_69214.png	실현하다 

In [ ]:
 %cd /content/drive/MyDrive/deep-text-recognition-benchmark

!python3 test.py \
    --eval_data data_lmdb/train --benchmark_all_eval \
    --Transformation None --FeatureExtraction 'VGG' --SequenceModeling "BiLSTM" --Prediction "CTC" \
    --saved_model saved_models/None-VGG-BiLSTM-CTC-Seed1111/best_accuracy.pth